In [1]:
# !pip install timm==1.0.9
# !pip install albumentations==1.4.14
# !pip install torcheval==0.0.7
# !pip install pandas==2.2.2
# !pip install numpy==1.26.4

In [1]:
import sys, os, time, copy, gc
import torch
from torch import nn
from torch.utils.data import DataLoader
from pathlib import Path

import numpy as np
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
import multiprocessing as mp

from torcheval.metrics.functional import binary_auroc, multiclass_auroc

from sklearn.model_selection import StratifiedGroupKFold

import hashlib
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split

from PIL import Image
import torch.optim as optim

from collections import defaultdict




sys.path.append('../src')
from utils import set_seed, visualize_augmentations_positive, print_trainable_parameters
from models import setup_model
from training import fetch_scheduler, train_one_epoch, valid_one_epoch, run_training, get_nth_test_step
from models import ISICModel, ISICModelEdgnet, setup_model
from datasets import ISICDatasetSamplerW, ISICDatasetSampler, ISICDatasetSimple, ISICDatasetSamplerMulticlass, prepare_loaders
from augmentations import get_augmentations

/opt/conda/lib/python3.11/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.15 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# Set up device and random seed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

Using device: cuda
GPU: NVIDIA RTX A6000
Number of GPUs: 1


In [3]:
original_data_path = "../data/original"
original_root = Path('../data/original')

data_artifacts = "../data/artifacts"
os.makedirs(data_artifacts, exist_ok=True)

In [4]:
# Set the HDF5 file path
TRAIN_HDF5_FILE_PATH = original_root / 'train-image.hdf5'

train_path = original_root / 'train-metadata.csv'
df_train = pd.read_csv(train_path)
df_train["path"] = '../data/original/train-image/image/' + df_train['isic_id'] + ".jpg"
original_positive_cases = df_train['target'].sum()
original_total_cases = len(df_train)
original_positive_ratio = original_positive_cases / original_total_cases

print(f"Number of positive cases: {original_positive_cases}")
print(f"Number of negative cases: {original_total_cases - original_positive_cases}")
print(f"Ratio of negative to positive cases: {(original_total_cases - original_positive_cases) / original_positive_cases:.2f}:1")

/tmp/ipykernel_2851924/2927513113.py:5: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(train_path)


Number of positive cases: 393
Number of negative cases: 400666
Ratio of negative to positive cases: 1019.51:1


In [5]:
MODEL_NAME = "EVA" # "EDGENEXT"


CONFIG = {
    "seed": 42 if MODEL_NAME == 'EVA' else 1997,
    "epochs": 500,
    "img_size": 336 if MODEL_NAME == 'EVA' else 256,
    "train_batch_size": 32,
    "valid_batch_size": 64,
    "learning_rate": 1e-4,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-6,
    "T_max": 2000,
    "weight_decay": 1e-6,
    "fold" : 0,
    "n_fold": 5,
    "n_accumulate": 1,
    "group_col": 'patient_id',
    "device": device
}

model_name = "eva02_small_patch14_336.mim_in22k_ft_in1k" if MODEL_NAME == 'EVA' else "edgenext_base.in21k_ft_in1k"
checkpoint_path = None


if MODEL_NAME == 'EVA':
    ISICModelPrep = ISICModel
else:
    ISICModelPrep = ISICModelEdgnet

In [6]:
data_transforms = get_augmentations(CONFIG)

/opt/conda/lib/python3.11/site-packages/pydantic/main.py:193: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  self.__pydantic_validator__.validate_python(data, self_instance=self)


In [7]:
def criterion(outputs, targets):
    return nn.BCELoss()(outputs, targets)

In [8]:
# synthetic_custom_data = f"../data/artifacts/syntetic_custom_base_{CONFIG['seed']}"
# os.makedirs(synthetic_custom_data, exist_ok=True)

# tsp = StratifiedGroupKFold(2, shuffle=True, random_state=CONFIG['seed'])
# metrics_ev_df = []
# test_forecast = []
# val_forecast = []
# for fold_n, (train_index, val_index) in enumerate(tsp.split(df_train, y=df_train.target, groups=df_train[CONFIG["group_col"]])):
#     fold_df_train = df_train.iloc[train_index].reset_index(drop=True)
#     fold_df_valid = df_train.iloc[val_index].reset_index(drop=True)
#     synthetic_custom_data_pr = os.path.join(synthetic_custom_data, str(fold_n))
#     os.makedirs(synthetic_custom_data_pr, exist_ok=True)

#     for fn in fold_df_train[fold_df_train.target==1].isic_id.values:
#         if fn not in images_to_include:
#             continue
#         img = Image.open(os.path.join('../data/original/train-image/image', fn + ".jpg"))
#         img.save(os.path.join(synthetic_custom_data_pr, fn + ".png"))
    

In [9]:
folder_name = f"../models/oof_{{MODEL_NAME.lower()}}_base"
os.makedirs(folder_name, exist_ok=True)

In [10]:
def get_metrics(drop_path_rate, drop_rate, models_folder, model_maker):
    tsp = StratifiedGroupKFold(5, shuffle=True, random_state=CONFIG['seed'])
    results_list = []
    fold_df_valid_list = []
    for fold_n, (train_index, val_index) in enumerate(tsp.split(df_train, y=df_train.target, groups=df_train[CONFIG["group_col"]])):
        fold_df_train = df_train.iloc[train_index].reset_index(drop=True)
        fold_df_valid = df_train.iloc[val_index].reset_index(drop=True)
        set_seed(CONFIG['seed'])
        model = setup_model(model_name, drop_path_rate=drop_path_rate, drop_rate=drop_rate, model_maker=model_maker)
        print_trainable_parameters(model)

        train_loader, valid_loader = prepare_loaders(fold_df_train, fold_df_valid, CONFIG, data_transforms)
    
        optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], 
                           weight_decay=CONFIG['weight_decay'])
        scheduler = fetch_scheduler(optimizer, CONFIG)
    
        model, history = run_training(
            train_loader, valid_loader,
            model, optimizer, scheduler,
            device=CONFIG['device'],
            num_epochs=CONFIG['epochs'],
            CONFIG=CONFIG, 
            tolerance_max=20,
            test_every_nth_step=lambda x: 5,
            seed=CONFIG['seed'])
        torch.save(model.state_dict(), os.path.join(models_folder, f"model__{fold_n}"))
        results_list.append(np.max(history['Valid Kaggle metric']))

        val_epoch_loss, val_epoch_auroc, val_epoch_custom_metric, tmp_predictions_all, tmp_targets_all = valid_one_epoch(
            model, 
            valid_loader, 
            device=CONFIG['device'], 
            epoch=1, 
            optimizer=optimizer, 
            criterion=criterion, 
            use_custom_score=True,
            metric_function=binary_auroc, 
            num_classes=1,
            return_preds=True)

        fold_df_valid['tmp_targets_all'] = tmp_targets_all
        fold_df_valid['tmp_predictions_all'] = tmp_predictions_all
        fold_df_valid['fold_n'] = fold_n
        fold_df_valid_list.append(fold_df_valid)
    fold_df_valid_list = pd.concat(fold_df_valid_list).reset_index(drop=True)
    return results_list, fold_df_valid_list

In [ ]:
base_metrics, oof_forecasts = get_metrics(drop_path_rate=0, drop_rate=0, models_folder=folder_name, model_maker=ISICModelPrep)
oof_forecasts.to_parquet(f'../data/artifacts/oof_forecasts_{MODEL_NAME.lower()}_base.parquet')

trainable params: 21744385 || all params: 21744385 || trainable%: 100.00


100%|██████████| 19/19 [00:08<00:00,  2.14it/s, Epoch=1, LR=0.0001, Train_Auroc=0.582, Train_Loss=0.754]


100%|██████████| 19/19 [00:06<00:00,  2.72it/s, Epoch=2, LR=9.99e-5, Train_Auroc=0.751, Train_Loss=0.676]


100%|██████████| 19/19 [00:07<00:00,  2.71it/s, Epoch=3, LR=9.98e-5, Train_Auroc=0.825, Train_Loss=0.546]


100%|██████████| 19/19 [00:07<00:00,  2.68it/s, Epoch=4, LR=9.96e-5, Train_Auroc=0.853, Train_Loss=0.523]


...

100%|██████████| 1214/1214 [05:03<00:00,  4.01it/s, Epoch=110, LR=3.42e-6, Valid_Auroc=0.521, Valid_Loss=0.387]
/opt/conda/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:1063: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  _warn_get_lr_called_within_step(self)


100%|██████████| 20/20 [00:08<00:00,  2.47it/s, Epoch=111, LR=3.93e-6, Train_Auroc=0.979, Train_Loss=0.194]


100%|██████████| 20/20 [00:07<00:00,  2.56it/s, Epoch=112, LR=4.48e-6, Train_Auroc=0.983, Train_Loss=0.181]


100%|██████████| 20/20 [00:07<00:00,  2.50it/s, Epoch=113, LR=5.07e-6, Train_Auroc=0.972, Train_Loss=0.21] 


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
1

# Train with synthetic data

In [ ]:
def get_metrics_synth(drop_path_rate, drop_rate, models_folder, model_maker, synth_path = "../data/artifacts/syntetic_base_folds_final"):
    tsp = StratifiedGroupKFold(5, shuffle=True, random_state=CONFIG['seed'])
    results_list = []
    fold_df_valid_list = []
    for fold_n, (train_index, val_index) in enumerate(tsp.split(df_train, y=df_train.target, groups=df_train[CONFIG["group_col"]])):
        fold_df_train = df_train.iloc[train_index].reset_index(drop=True)
        fold_df_valid = df_train.iloc[val_index].reset_index(drop=True)
        set_seed(CONFIG['seed'])
        model = setup_model(model_name, drop_path_rate=drop_path_rate, drop_rate=drop_rate, model_maker=model_maker)
        print_trainable_parameters(model)

        synthetic_Df = pd.concat([
            pd.DataFrame({
                "path": glob.glob(f"{synth_path}/{CONFIG['seed']}/{fold_n}/hr/**.png")
            }),
            pd.DataFrame({
                "path": glob.glob(f"{synth_path}/{CONFIG['seed']}/{fold_n}/lr/**.png")
            })
        ])
        
        synthetic_Df['weight'] = 1
        synthetic_Df['target'] = synthetic_Df.path.apply(lambda x: int(x.split('___')[1].split('.')[0]))

        synthetic_Df = pd.concat([
            synthetic_Df, fold_df_train[['path', 'target', 'weight']].reset_index(drop=True)
        ]).reset_index(drop=True)

        
        train_loader, valid_loader = prepare_loaders(synthetic_Df, fold_df_valid, CONFIG, data_transforms)
    
        optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], 
                           weight_decay=CONFIG['weight_decay'])
        scheduler = fetch_scheduler(optimizer, CONFIG)
    
        model, history = run_training(
            train_loader, valid_loader,
            model, optimizer, scheduler,
            device=CONFIG['device'],
            num_epochs=CONFIG['epochs'],
            CONFIG=CONFIG, 
            tolerance_max=20,
            test_every_nth_step=lambda x: 5,
            seed=CONFIG['seed'])
        torch.save(model.state_dict(), os.path.join(models_folder, f"model__{fold_n}"))
        results_list.append(np.max(history['Valid Kaggle metric']))

        val_epoch_loss, val_epoch_auroc, val_epoch_custom_metric, tmp_predictions_all, tmp_targets_all = valid_one_epoch(
            model, 
            valid_loader, 
            device=CONFIG['device'], 
            epoch=1, 
            optimizer=optimizer, 
            criterion=criterion, 
            use_custom_score=True,
            metric_function=binary_auroc, 
            num_classes=1,
            return_preds=True)

        fold_df_valid['tmp_targets_all'] = tmp_targets_all
        fold_df_valid['tmp_predictions_all'] = tmp_predictions_all
        fold_df_valid['fold_n'] = fold_n
        fold_df_valid_list.append(fold_df_valid)
    fold_df_valid_list = pd.concat(fold_df_valid_list).reset_index(drop=True)
    return results_list, fold_df_valid_list

In [ ]:
folder_name = f"../models/oof_{{MODEL_NAME.lower()}}_base__synth"
os.makedirs(folder_name, exist_ok=True)

In [ ]:
base_metrics, oof_forecasts = get_metrics(drop_path_rate=0, drop_rate=0, models_folder=folder_name, model_maker=ISICModelPrep)
oof_forecasts.to_parquet(f'../data/artifacts/oof_forecasts_{MODEL_NAME.lower()}_base__synth.parquet')